In [3]:
!pip install autodistill autodistill-grounded-sam autodistill-yolov8 supervision
!pip install ultralytics
!pip install tensorflow
!pip install keras
!pip install kaggle
!pip install opencv-python
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [4]:
import os
import multiprocessing
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from autodistill_grounded_sam import GroundedSAM
from autodistill.detection import CaptionOntology
from autodistill_yolov8 import YOLOv8
from ultralytics import YOLO
import supervision as sv

In [8]:
!mkdir -p /kaggle/working/dataset /kaggle/working/dataset/images /kaggle/working/dataset/labels
!cp -r /kaggle/input/sports-balls-multiclass-image-classification/train /kaggle/input/sports-balls-multiclass-image-classification/test /kaggle/working/dataset/images 

In [9]:
def annotate_with_autodistill(dataset_name, data_split):
    """Annotates images using Autodistill for a specific dataset and split."""

    input_folder = os.path.join("/content/dataset/images", data_split, dataset_name)
    output_folder = os.path.join("/content/dataset/labels", data_split, dataset_name)

    # Define your ontology (mapping of class names to prompts for GroundedSAM)
    ontology = CaptionOntology({"american_football" : "american_football", "baseball": "baseball", "basketball": "basketball", "billiard_ball": "billiard_ball", "bowling_ball": "bowling_ball",
    "cricket_ball": "cricket_ball", "football": "football", "golf_ball": "golf_ball", "field_hockey_ball": "field_hockey_ball", "hockey_puck": "hockey_puck",
    "rugby_ball": "rugby_ball", "shuttlecock": "shuttlecock", "table_tennis_ball": "table_tennis_ball", "tennis_ball": "tennis_ball", "volleyball": "volleyball"})  # Adjust the ontology as needed

    # Initialize the GroundedSAM base model
    base_model = GroundedSAM(ontology=ontology)

    # Annotate your images
    base_model.label(
        input_folder=input_folder,
        output_folder=output_folder,
    )

    # Initialize the GroundedSAM base model
    base_model = GroundedSAM(ontology=ontology)

    # Annotate your images
    base_model.label(
        input_folder=input_folder,
        output_folder=output_folder,
    )

In [ ]:
!wandb login d336a0192867d2bfcda67821016ac99bc6a74924

In [10]:
# Annotate for train and test splits
for dataset_name in [
    "american_football", "baseball", "basketball", "billiard_ball", "bowling_ball",
    "cricket_ball", "football", "golf_ball", "field_hockey_ball", "hockey_puck",
    "rugby_ball", "shuttlecock", "table_tennis_ball", "tennis_ball", "volleyball"
]:  # Replace with your dataset names
    for data_split in ["train", "test"]:
        annotate_with_autodistill(dataset_name, data_split)

trying to load grounding dino directly
downloading dino model weights


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)


final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Labeling images: 0it [00:00, ?it/s]


Labeled dataset created - ready for distillation.
trying to load grounding dino directly
final text_encoder_type: bert-base-uncased


Labeling images: 0it [00:00, ?it/s]


Labeled dataset created - ready for distillation.
trying to load grounding dino directly
final text_encoder_type: bert-base-uncased


KeyboardInterrupt: 

In [ ]:
def train_yolov8(dataset_name):
    """Trains YOLOv8 for a specific dataset using annotations from Autodistill."""

    data_yaml_path = os.path.join("/content/dataset/labels/train", dataset_name, "data.yaml")

    # Load the pre-trained YOLOv8 model (or specify your custom model path)
    model = YOLO('yolov8n.pt') 

    # Train the YOLOv11 model using the annotated data
    results = model.train(
        data=data_yaml_path,
        epochs=100,  # Adjust the number of epochs as needed
    )

    return model  # Return the trained model

In [ ]:
# Train YOLOv11 models for each dataset
trained_models = {}
for dataset_name in [
    "american_football", "baseball", "basketball", "billiard_ball", "bowling_ball",
    "cricket_ball", "football", "golf_ball", "field_hockey_ball", "hockey_puck",
    "rugby_ball", "shuttlecock", "table_tennis_ball", "tennis_ball", "volleyball"
]:  # Replace with your dataset names
    trained_models[dataset_name] = train_yolov8(dataset_name)